In [1]:
import pandas as pd
import numpy as np

from scipy.stats import ks_2samp
#import statsmodels.stats.multitest as multi

import os.path

from helpers import bh

In [2]:
folder = '/Users/zivaskof/Documents/MATLAB/mag/fluxSampels/tInitSampels'

In [5]:
filenames = os.listdir(folder)
filenames.sort()

for i in range(1, len(filenames), 2):  # Iterate with a step of 2 to get consecutive pairs
    file_control = filenames[i]
    file_kd = filenames[i + 1]

    print(file_control)
    print(file_kd)
    
    df_control = pd.read_csv(os.path.join(folder, file_control))
    df_kd = pd.read_csv(os.path.join(folder, file_kd))
    
    reactions = sorted(list(set(df_control.columns) | set(df_kd.columns)))
    len(reactions) # število reakcij
    
    df = pd.DataFrame(columns=['reaction', 'FC', 'p', 'q', 'enrichment', 'changed'])
    df['reaction']=reactions

    n_samples = df_control.shape[0]

    # sprehodimo se čez vse reakcije
    for reaction in reactions:
        if reaction in df_control.columns:
            control = df_control[reaction].values
        else:
            # če reakcije ni v kontrolni skupini, ji pripišemo same ničle
            control = np.zeros(n_samples)

        if reaction in df_kd.columns:
            kd = df_kd[reaction].values
        else:
            # če reakcije ni v kd skupini, ji pripišemo same ničle
            kd = np.zeros(n_samples)

        # iztračunamo sredino za kontrolo in kd
        mean_control = np.mean(control)
        mean_kd = np.mean(kd)

        # izračunamo FC - fold change in signifikanco z uporabo 2 sample Kolmogorov-Smirnov testa
        if mean_control != 0 or mean_kd != 0:
            FC = mean_kd-mean_control/(abs(mean_kd + mean_control))
            p = ks_2samp(control,kd)[1]
        else:
            FC = 0
            p = 1     

        df.loc[df['reaction']==reaction, 'FC'] = FC
        df.loc[df['reaction']==reaction, 'p'] = p


    # korigiramo p vrednosti za večkratno testiranje - FDR korekcija
    df['q'] = bh(df['p'])

    # signifikanca zahteva vsaj 10-kratno up-/down-regulacijo
    df.loc[(df['FC'] >= 0.82) & (df['q'] < 0.05),'enrichment'] = 1
    df.loc[(df['FC'] <= -0.82) & (df['q'] < 0.05),'enrichment'] = -1
    df.loc[~df['enrichment'].isna(),'changed'] = 1
    df = df.fillna(0)
    
    minus_ena = df[df.enrichment == -1]
    print(minus_ena)
    ena = df[df.enrichment == 1]
    print(ena)
    nic = df[df.enrichment == 0]
    print(nic)
    
    output_path = '/Users/zivaskof/Documents/MATLAB/mag/reaction_dif/tInitReactions'
    extracted_string = file_control.split("_", maxsplit=3)[-1][:-5]
    new_file_name = 'iMAT_reaction_diff_' + extracted_string + ".csv"
    print(new_file_name)
    csv_file_path = os.path.join(output_path, new_file_name)
    df.to_csv(csv_file_path, index=False) 
    print("------------------------------")

tINIT_flux_sample_KM_001N.csv
tINIT_flux_sample_KM_001T.csv
      reaction        FC              p              q  enrichment  changed
5     MAR00018 -1.000000  3.580820e-149  6.520862e-149          -1        1
11    MAR00040 -1.220460   0.000000e+00   0.000000e+00          -1        1
12    MAR00047 -1.195181   0.000000e+00   0.000000e+00          -1        1
13    MAR00048 -1.195181   0.000000e+00   0.000000e+00          -1        1
30    MAR00101 -1.000000  2.386293e-165  4.509677e-165          -1        1
...        ...       ...            ...            ...         ...      ...
6014  MAR13012 -0.964294  8.575383e-250  2.062930e-249          -1        1
6051  MAR13081 -0.862338  9.185171e-122  1.598167e-121          -1        1
6052  MAR13082 -5.207502  6.118730e-288  1.639332e-287          -1        1
6053  MAR13083 -1.000000  2.523014e-285  6.691567e-285          -1        1
6054  MAR13086 -5.207502  6.118730e-288  1.639332e-287          -1        1

[899 rows x 6 columns]
    

      reaction          FC    p    q  enrichment  changed
14    MAR00045   -1.000000  0.0  0.0          -1        1
37    MAR00125 -273.497258  0.0  0.0          -1        1
812   MAR01264 -703.649350  0.0  0.0          -1        1
814   MAR01266 -661.279994  0.0  0.0          -1        1
825   MAR01285 -248.333983  0.0  0.0          -1        1
...        ...         ...  ...  ...         ...      ...
5528  MAR12267  -20.951080  0.0  0.0          -1        1
5531  MAR12270  -21.394514  0.0  0.0          -1        1
5552  MAR12295  -20.294262  0.0  0.0          -1        1
5558  MAR12301  -37.473352  0.0  0.0          -1        1
5563  MAR12306  -39.097300  0.0  0.0          -1        1

[243 rows x 6 columns]
      reaction          FC    p    q  enrichment  changed
8     MAR00025  466.371268  0.0  0.0           1        1
9     MAR00030  466.371268  0.0  0.0           1        1
10    MAR00031   51.679828  0.0  0.0           1        1
11    MAR00032   51.679828  0.0  0.0           1

      reaction          FC    p    q  enrichment  changed
16    MAR00045   -1.000000  0.0  0.0          -1        1
767   MAR01264 -519.771235  0.0  0.0          -1        1
769   MAR01266 -476.970898  0.0  0.0          -1        1
873   MAR01621 -480.944006  0.0  0.0          -1        1
967   MAR01802 -479.258562  0.0  0.0          -1        1
...        ...         ...  ...  ...         ...      ...
4450  MAR09902 -667.643961  0.0  0.0          -1        1
4452  MAR09905 -599.793438  0.0  0.0          -1        1
5039  MAR11395 -718.541805  0.0  0.0          -1        1
5052  MAR11433  -75.809878  0.0  0.0          -1        1
5089  MAR12022 -576.072464  0.0  0.0          -1        1

[213 rows x 6 columns]
      reaction          FC    p    q  enrichment  changed
9     MAR00025  508.947106  0.0  0.0           1        1
10    MAR00030  508.947106  0.0  0.0           1        1
11    MAR00031   88.844166  0.0  0.0           1        1
12    MAR00032   88.844166  0.0  0.0           1

      reaction          FC    p    q  enrichment  changed
16    MAR00045   -1.000000  0.0  0.0          -1        1
39    MAR00125 -274.776277  0.0  0.0          -1        1
736   MAR01264 -463.992249  0.0  0.0          -1        1
738   MAR01266 -240.928380  0.0  0.0          -1        1
809   MAR01528  -32.875156  0.0  0.0          -1        1
...        ...         ...  ...  ...         ...      ...
4223  MAR09900 -403.086706  0.0  0.0          -1        1
4225  MAR09902 -728.451512  0.0  0.0          -1        1
4226  MAR09905 -858.495506  0.0  0.0          -1        1
4304  MAR11395 -691.213856  0.0  0.0          -1        1
4318  MAR11433 -104.113383  0.0  0.0          -1        1

[166 rows x 6 columns]
      reaction          FC    p    q  enrichment  changed
9     MAR00025  421.748199  0.0  0.0           1        1
10    MAR00030  421.748199  0.0  0.0           1        1
11    MAR00031   83.908253  0.0  0.0           1        1
12    MAR00032   83.908253  0.0  0.0           1

      reaction         FC             p             q  enrichment  changed
13    MAR00047  -6.838831  0.000000e+00  0.000000e+00          -1        1
14    MAR00048  -6.838831  0.000000e+00  0.000000e+00          -1        1
15    MAR00050 -15.505964  0.000000e+00  0.000000e+00          -1        1
16    MAR00053  -1.000000  0.000000e+00  0.000000e+00          -1        1
18    MAR00059 -84.138128  2.725103e-58  4.990059e-58          -1        1
...        ...        ...           ...           ...         ...      ...
5939  MAR12917  -1.000000  0.000000e+00  0.000000e+00          -1        1
5947  MAR12955  -1.000000  0.000000e+00  0.000000e+00          -1        1
5993  MAR13080  -1.000000  0.000000e+00  0.000000e+00          -1        1
5995  MAR13082  -0.997565  0.000000e+00  0.000000e+00          -1        1
5996  MAR13086  -0.997565  0.000000e+00  0.000000e+00          -1        1

[1014 rows x 6 columns]
      reaction          FC              p              q  enrichment  chang

...

      reaction          FC              p              q  enrichment  changed
6     MAR00015   -1.000000   0.000000e+00   0.000000e+00          -1        1
28    MAR00076   -1.000000  1.211020e-203  2.805978e-203          -1        1
31    MAR00084   -1.000000   0.000000e+00   0.000000e+00          -1        1
36    MAR00096   -1.000000   0.000000e+00   0.000000e+00          -1        1
38    MAR00102 -148.135476  1.078246e-289  2.986838e-289          -1        1
...        ...         ...            ...            ...         ...      ...
6326  MAR12727 -495.712838   0.000000e+00   0.000000e+00          -1        1
6355  MAR12781   -1.000000   0.000000e+00   0.000000e+00          -1        1
6376  MAR12844   -1.226737   1.214603e-62   2.163396e-62          -1        1
6389  MAR12920   -1.000000   0.000000e+00   0.000000e+00          -1        1
6399  MAR12965   -1.000000   0.000000e+00   0.000000e+00          -1        1

[1312 rows x 6 columns]
      reaction          FC             

      reaction          FC              p              q  enrichment  changed
6     MAR00015   -1.000000   0.000000e+00   0.000000e+00          -1        1
20    MAR00059   -1.610758   6.315156e-52   1.096434e-51          -1        1
21    MAR00060 -461.631794   0.000000e+00   0.000000e+00          -1        1
22    MAR00064 -227.293710   0.000000e+00   0.000000e+00          -1        1
28    MAR00079   -2.890311  6.118730e-288  1.873173e-287          -1        1
...        ...         ...            ...            ...         ...      ...
6580  MAR12956   -1.023699   0.000000e+00   0.000000e+00          -1        1
6582  MAR12965   -5.746328  2.205050e-140  4.491343e-140          -1        1
6593  MAR13022   -1.023699   0.000000e+00   0.000000e+00          -1        1
6600  MAR13040   -1.000000  3.044463e-194  7.184977e-194          -1        1
6627  MAR13075 -481.972183   0.000000e+00   0.000000e+00          -1        1

[971 rows x 6 columns]
      reaction          FC              

      reaction          FC              p              q  enrichment  changed
5     MAR00018   -1.000000  1.107250e-180  2.550117e-180          -1        1
16    MAR00054   -1.000000   0.000000e+00   0.000000e+00          -1        1
31    MAR00095   -1.000000  1.333274e-262  3.895312e-262          -1        1
38    MAR00111 -213.095652   0.000000e+00   0.000000e+00          -1        1
41    MAR00115 -128.063704  1.083476e-297  3.469657e-297          -1        1
...        ...         ...            ...            ...         ...      ...
6139  MAR12539  -19.966501  5.247906e-142  1.089896e-141          -1        1
6143  MAR12551   -1.000000   0.000000e+00   0.000000e+00          -1        1
6146  MAR12557   -1.000000   0.000000e+00   0.000000e+00          -1        1
6241  MAR12917  -19.966501  5.247906e-142  1.089896e-141          -1        1
6246  MAR12939  -19.966501  5.247906e-142  1.089896e-141          -1        1

[961 rows x 6 columns]
      reaction          FC              

      reaction         FC              p              q  enrichment  changed
4     MAR00015  -1.000000  1.724145e-240  4.602965e-240          -1        1
11    MAR00039  -1.000000   0.000000e+00   0.000000e+00          -1        1
13    MAR00047  -2.701327  2.732385e-158  5.775996e-158          -1        1
14    MAR00048  -2.701327  2.732385e-158  5.775996e-158          -1        1
16    MAR00053 -35.094512   0.000000e+00   0.000000e+00          -1        1
...        ...        ...            ...            ...         ...      ...
5918  MAR12352  -1.000000  2.198901e-162  4.685344e-162          -1        1
5951  MAR12432  -1.000000  1.863595e-291  5.965641e-291          -1        1
5962  MAR12461  -1.000000   0.000000e+00   0.000000e+00          -1        1
6053  MAR12780  -1.000000  1.863595e-291  5.965641e-291          -1        1
6084  MAR12915  -1.000000  1.863595e-291  5.965641e-291          -1        1

[1026 rows x 6 columns]
      reaction          FC              p          

IndexError: list index out of range